In [50]:
import pandas as pd
import numpy as np
import requests
import os
import json
from pprint import pprint

In [51]:
# Importing the df from city_bikes.ipynb
city_bikes_path = r"C:\Users\Parkavi\LHL\Statistical-Modelling-with-Python_Project\city_bikes_data.csv"

# Read the CSV file into a DataFrame
city_bikes_df = pd.read_csv(city_bikes_path)

In [52]:
city_bikes_df

,city,name,latitude,longitude,slots,empty_slots,no_of_bikes
0,Vancouver,10th & Cambie,49.262487,-123.114397,36,34,2
1,Vancouver,Yaletown-Roundhouse Station,49.274566,-123.121817,16,8,7
2,Vancouver,Dunsmuir & Beatty,49.279764,-123.110154,26,11,14
3,Vancouver,12th & Yukon (City Hall),49.260599,-123.113504,16,11,5
4,Vancouver,8th & Ash,49.264215,-123.117772,16,9,6
...,...,...,...,...,...,...,...
239,Vancouver,Cordova & Granville,49.285670,-123.112543,18,7,11
240,Vancouver,22nd & Main,49.250940,-123.101306,14,14,0
241,Vancouver,PNE - Hastings & Windermere,49.280977,-123.035969,40,25,15
242,Vancouver,1st & Fir,49.270783,-123.141564,20,3,17


# Foursquare

In [53]:
api_key = os.environ["FOURSQUARE_API_KEY"]

url = "https://api.foursquare.com/v3/places/search"


#params = {
  	#"query": "restaurant",
    #"location": "Vancouver",
    #"radius": "1000"
  #	}

headers = {
    "Accept": "application/json",
    "Authorization": os.environ["FOURSQUARE_API_KEY"]
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"4b4a3c82f964a520168026e3","categories":[{"id":16039,"name":"Urban Park","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_","suffix":".png"}}],"chains":[],"distance":411,"geocodes":{"drop_off":{"latitude":48.420937,"longitude":-123.366274},"main":{"latitude":48.420842,"longitude":-123.365713},"roof":{"latitude":48.420842,"longitude":-123.365713}},"link":"/v3/places/4b4a3c82f964a520168026e3","location":{"address":"Bordered by Douglas St","country":"CA","cross_street":"at Circle Dr.","formatted_address":"Bordered by Douglas St (at Circle Dr.), Victoria BC","locality":"Victoria","region":"BC"},"name":"Beacon Hill Park","related_places":{"children":[{"fsq_id":"4f85a7c6e4b041623e92a894","name":"Goodacre Lake"},{"fsq_id":"4beb0de2415e20a179d3e5bb","name":"Beacon Hill Children's Farm"},{"fsq_id":"5bea107975eee4002c826d6f","name":"The Moss Lady"},{"fsq_id":"62992e087524756513e676b9","name":"Queen’s Lake"},{"fsq_id":"4e839cd46c2508dd35012874","n

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [54]:
param= []
# Iterate through rows using iterrows()
for index, row in city_bikes_df.iterrows():
    ll = str(row['latitude']) + "," + str(row['longitude'])
    param.append({
       "ll": ll,
       "radius": 1000,
    })
param

[{'ll': '49.262487,-123.114397', 'radius': 1000},
 {'ll': '49.274566,-123.121817', 'radius': 1000},
 {'ll': '49.279764,-123.110154', 'radius': 1000},
 {'ll': '49.260599,-123.113504', 'radius': 1000},
 {'ll': '49.264215,-123.117772', 'radius': 1000},
 {'ll': '49.270877,-123.115103', 'radius': 1000},
 {'ll': '49.26893,-123.11813', 'radius': 1000},
 {'ll': '49.271721,-123.104207', 'radius': 1000},
 {'ll': '49.287214,-123.11287', 'radius': 1000},
 {'ll': '49.282409,-123.118541', 'radius': 1000},
 {'ll': '49.287097,-123.118551', 'radius': 1000},
 {'ll': '49.273471,-123.118016', 'radius': 1000},
 {'ll': '49.282758,-123.122092', 'radius': 1000},
 {'ll': '49.28602,-123.116624', 'radius': 1000},
 {'ll': '49.279739,-123.116251', 'radius': 1000},
 {'ll': '49.277483,-123.114279', 'radius': 1000},
 {'ll': '49.275679,-123.116963', 'radius': 1000},
 {'ll': '49.282759,-123.106955', 'radius': 1000},
 {'ll': '49.288444,-123.139203', 'radius': 1000},
 {'ll': '49.289124,-123.12196', 'radius': 1000},
 {'ll

In [55]:
## Filtering with rating, price, and location
# 13065- Restaurant 11044- ATM 16032- Park
#url = "https://api.foursquare.com/v3/places/search?categories=13065%2C13032%2C12013&fields=name%2Crating%2Cprice%2Clocation%2Cpopularity%2Ccategories%2Cgeocodes"
url= "https://api.foursquare.com/v3/places/search?categories=13065%2C11044%2C16032&fields=name%2Crating%2Cprice%2Clocation%2Cdistance%2Ccategories%2Cgeocodes"

#GET Response from url
#Create list to store Foursquare data from GET request
#Adding the latitude and longitude pair to to the json file to keep track of which bike stations yielded which results

Foursquare_data= []

for row in param:
    response = requests.request("GET", url, params=row, headers=headers)
    Foursquare_json = response.json()
    Foursquare_data.append([row['ll'], Foursquare_json])

In [56]:
#Confirm that for loop iterated through all bike stations
print(len(Foursquare_data))
#print(Foursquare_data)

244


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [42]:
#Create empty lists to add to dataframe

Bike_Station_Coordinates = []
Name = []
Category1 = []
Category2 = []
Category3 = []
Distance =[]
Latitude = []
Longitude = []
Locality = []
Price = []
Rating = []

In [43]:
#Iterate through Foursquare_data made from GET request. 
#Add appropriate items to each empty list for each column 
#Try, except added so that missing values can be documented in the excel sheet

for store in Foursquare_data:
    businesses = store[1]['results']
    station_coordinates = store[0]
    for business in businesses:
        Bike_Station_Coordinates.append(station_coordinates)
        Name.append(business['name'])
        try:
            Category1.append(business['categories'][0]['name'])
        except IndexError as e:
            Category1.append(None)
        try:
            Category2.append(business['categories'][1]['name'])
        except IndexError as e:
            Category2.append(None)
        try:
            Category3.append(business['categories'][2]['name'])
        except IndexError as e:
            Category3.append(None)        
        try:
            Distance.append(business['distance'])
        except (KeyError, IndexError) as e:
            Distance.append(None)
        Latitude.append(business['geocodes']['main']['latitude'])
        Longitude.append(business['geocodes']['main']['longitude'])
        try:
            Locality.append(business['location']['locality'])
        except (KeyError, IndexError) as e:
            Locality.append(None)       
        try:
            Price.append(business['price'])
        except (KeyError, IndexError) as e:
            Price.append(None)
        try:
            Rating.append(business['rating'])
        except (KeyError, IndexError) as e:
            Rating.append(None)
        

In [45]:
#Check to see if all the lists are the same length

print(len(Bike_Station_Coordinates))
print(len(Name))
print(len(Category1))
print(len(Category2))
print(len(Category3))
print(len(Distance))
print(len(Latitude))
print(len(Longitude))
print(len(Locality))
print(len(Price))
print(len(Rating))

2436
2436
2436
2436
2436
2436
2436
2436
2436
2436
2436


Put your parsed results into a DataFrame

In [97]:
#Create Foursquare dataframe with the parsed lists 

Foursquare_df = pd.DataFrame({  
    
    'bike_station_coordinates': Bike_Station_Coordinates,
    'name': Name,
    'category1': Category1,
    'category2': Category2,
    'category3': Category3,
    'distance': Distance,
    'latitude': Latitude,
    'longitude': Longitude,
    'locality': Locality,    
    'price': Price,
    'rating': Rating
})

#Preview the dataframe
Foursquare_df.head()

,bike_station_coordinates,name,category1,category2,category3,distance,latitude,longitude,locality,price,rating
0,"49.262487,-123.114397",Marulilu Cafe,Café,Japanese Restaurant,None,96,49.263328,-123.114020,Vancouver,2.0,8.2
1,"49.262487,-123.114397",Aperture Coffee Bar,Café,Coffee Shop,Restaurant,299,49.263292,-123.110529,Vancouver,1.0,8.7
2,"49.262487,-123.114397",Milano Coffee,Coffee Shop,Restaurant,None,430,49.263724,-123.108701,Vancouver,2.0,9.0
3,"49.262487,-123.114397",Caffe Cittadella,Coffee Shop,Restaurant,None,336,49.264730,-123.116978,Vancouver,2.0,8.6
4,"49.262487,-123.114397",Elysian Coffee Broadway,Coffee Shop,Restaurant,None,225,49.263077,-123.117243,Vancouver,1.0,8.1
...,...,...,...,...,...,...,...,...,...,...,...
2431,"49.264019,-123.209176",Mix the Bakery,Bakery,Pizzeria,Fast Food Restaurant,157,49.263658,-123.207055,Vancouver,1.0,6.7
2432,"49.264019,-123.209176",Subway,Deli,Fast Food Restaurant,None,303,49.263860,-123.204964,Vancouver,1.0,6.5
2433,"49.264019,-123.209176",Pizza Hut,Pizzeria,Restaurant,None,130,49.263728,-123.210465,Vancouver,1.0,6.3
2434,"49.264019,-123.209176",Gold Train Express,Vietnamese Restaurant,None,None,75,49.263702,-123.210139,Vancouver,2.0,6.0


In [47]:
#Check the dimensions of the dataframe
print(Foursquare_df.shape)

(2436, 11)


In [57]:
#Save the dataframe as a csv file  #saved in C://Users/srini ----> ('../foursquare_data.csv') and C://Users/srini/Statistical_project_python
Foursquare_df.to_csv('foursquare_data_latest.csv', index=False)

# Yelp

In [137]:
###################YELP API #########################
os.environ['YELP_API_KEY'] = "hSkpCytcQ9iyk4ugMxR1hqsbs8LQpJUxbiqdX7QKzu5c70giSfnvmWSFT5lOS--JWHAz4P0OciGCkrkPe2IRT3JtJOyGxMt4hHj8ylJITYl8OqwSiavG21nEmq_0ZHYx"
YELP_KEY = os.environ['YELP_API_KEY']

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [138]:
#GET request to YELP API
#Filtering with rating, price, radius, and college/universites

url = 'https://api.yelp.com/v3/businesses/search?location=Vancouver%2C%20BC&radius=1000&categories=restaurants&categories=banks&categories=parks&price=1&price=2&price=3&price=4'

# Create dictionary for headers
headers = {"Accept": "application/json"}
# Add key with our API KEY
headers['Authorization'] = f"Bearer {YELP_KEY}"

In [139]:
#Iterate through each bike station from city_bikes.csv
param=[]

#Iterate through rows using iterrows() with latitude, longitude as an interger pair.
#Format the latitude and longitude pair to add to yelp fetch url 
for index, row in city_bikes_df.iterrows():
    latitude= str(row['latitude']) 
    longitude= str(row['longitude'])
    ll= latitude + ','+ longitude
    param.append(ll)
print(param)

['49.262487,-123.114397', '49.274566,-123.121817', '49.279764,-123.110154', '49.260599,-123.113504', '49.264215,-123.117772', '49.270877,-123.115103', '49.26893,-123.11813', '49.271721,-123.104207', '49.287214,-123.11287', '49.282409,-123.118541', '49.287097,-123.118551', '49.273471,-123.118016', '49.282758,-123.122092', '49.28602,-123.116624', '49.279739,-123.116251', '49.277483,-123.114279', '49.275679,-123.116963', '49.282759,-123.106955', '49.288444,-123.139203', '49.289124,-123.12196', '49.28062,-123.12482', '49.279938,-123.127478', '49.273663,-123.127075', '49.28972,-123.144028', '49.285253,-123.134356', '49.283343,-123.131099', '49.284605,-123.108662', '49.279821,-123.10802', '49.278004,-123.105664', '49.279342,-123.101822', '49.273833,-123.114373', '49.28526,-123.141809', '49.281506,-123.120101', '49.276115,-123.114496', '49.272827,-123.147744', '49.269361,-123.13782', '49.2653,-123.13811', '49.262598,-123.138921', '49.25894,-123.139272', '49.258763,-123.115392', '49.266724,-12

In [140]:
#test case for 1 url 
response= requests.request("GET", url='https://api.yelp.com/v3/businesses/search?location=Vancouver%2C%20BC&radius=1000&categories=restaurants&categories=banks&categories=parks&price=1&price=2&price=3&price=4&latitude=49.262487&longitude=-123.114397',headers=headers)
print(response)

<Response [200]>


In [142]:
#url = 'https://api.yelp.com/v3/businesses/search?location=Vancouver%2C%20BC&radius=1000&categories=restaurants&categories=banks&categories=parks&price=1&price=2&price=3&price=4'

#GET Response from url
#Create list to store Foursquare data from GET request
#Adding the latitude and longitude pair to to the json file to keep track of which bike stations yielded which results

yelp_data= []

for row in param:
    response = requests.request("GET", url, params=row, headers=headers)
    yelp_json = response.json()
    yelp_data.append([row, yelp_json]) # try to remove row and just pass yelp_json and see if the same 4880 rows of data appears

In [123]:
#Verify yelp_data looks correct
print(len(yelp_data))

244


In [143]:
yelp_data[0]

['49.262487,-123.114397',
 {'businesses': [{'id': 'XAH2HpuUUtu7CUO26pbs4w',
    'alias': 'saku-vancouver-8',
    'name': 'Saku',
    'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/mKOSejpgcOkslnUydp0vSg/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/saku-vancouver-8?adjust_creative=3Dry9JG6h1oBCDayDIr53A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=3Dry9JG6h1oBCDayDIr53A',
    'review_count': 224,
    'categories': [{'alias': 'japanese', 'title': 'Japanese'}],
    'rating': 4.5,
    'coordinates': {'latitude': 49.2631006, 'longitude': -123.11667523250104},
    'transactions': [],
    'price': '$$',
    'location': {'address1': '548 W Broadway',
     'address2': '',
     'address3': None,
     'city': 'Vancouver',
     'zip_code': 'V5Z 1E9',
     'country': 'CA',
     'state': 'BC',
     'display_address': ['548 W Broadway', 'Vancouver, BC V5Z 1E9', 'Canada']},
    'phone': '+16044234600',
    'display_phone': '+1 604-423-4600',
    

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [144]:
#Make lists for each column that will be put in the dataframe. These will be parsed for in the yelp_data

yelp_bike_station_coordinates = []
yelp_name = []
yelp_latitude = []
yelp_longitude = []
yelp_rating = []
yelp_price = []
yelp_distance = []
yelp_category1 = []
yelp_category2 = []
yelp_category3 = []
yelp_review_count = []


In [145]:
#Parse the yelp_data to pull for specific parameters. Add them to appropriate lists made. 
for shop in yelp_data:
    businesses = shop[1]['businesses']
    station_coordinates = shop[0]
    for business in businesses:
        yelp_bike_station_coordinates.append(station_coordinates)
        yelp_name.append(business['name'])
        yelp_latitude.append(business['coordinates']['latitude'])
        yelp_longitude.append(business['coordinates']['longitude'])
        yelp_rating.append(business['rating'])
        yelp_price.append(business['price'])     
        yelp_distance.append(business['distance'])          
        yelp_category1.append(business['categories'][0]['title'])
        try:
            yelp_category2.append(business['categories'][1]['title'])
        except (KeyError, IndexError) as e:
            yelp_category2.append(None)
        try:
            yelp_category3.append(business['categories'][1]['title'])
        except (KeyError, IndexError) as e:
            yelp_category3.append(None)
        yelp_review_count.append(business['review_count'])

In [146]:
#Check to see if length of each list is the same 

print(len(yelp_bike_station_coordinates))
print(len(yelp_name))
print(len(yelp_latitude))
print(len(yelp_longitude))
print(len(yelp_rating))
print(len(yelp_price))

print(len(yelp_distance))
print(len(yelp_category1))
print(len(yelp_category2))
print(len(yelp_category3))
print(len(yelp_review_count))


4880
4880
4880
4880
4880
4880
4880
4880
4880
4880
4880


Put your parsed results into a DataFrame

In [147]:
#create dataframe with desired columns using the lists made
yelp_df = pd.DataFrame({
    'yelp_bike_station_coordinates': yelp_bike_station_coordinates,
    'name': yelp_name,
    'latitude': yelp_latitude,
    'longitude': yelp_longitude,
    'rating': yelp_rating,
    'price': yelp_price,    
    'distance': yelp_distance,
    'category_1': yelp_category1,
    'category_2': yelp_category2,
    'category_3': yelp_category3,
    'review_count': yelp_review_count
    })

#preview the dataframe 
#yelp_df.head()
yelp_df

,yelp_bike_station_coordinates,name,latitude,longitude,rating,price,distance,category_1,category_2,category_3,review_count
0,"49.262487,-123.114397",Saku,49.263101,-123.116675,4.5,$$,38.092128,Japanese,None,None,224
1,"49.262487,-123.114397",Yolks,49.263114,-123.116353,3.5,$$,29.354770,Coffee & Tea,Breakfast & Brunch,Breakfast & Brunch,348
2,"49.262487,-123.114397",Marulilu Cafe,49.263380,-123.114150,4.0,$$,153.789445,Cafes,None,None,286
3,"49.262487,-123.114397",La Taqueria Pinche Taco Shop,49.263559,-123.112736,4.0,$$,255.181363,Mexican,None,None,682
4,"49.262487,-123.114397",Hokkaido Ramen Santouka,49.263127,-123.116892,4.0,$$,50.822267,Noodles,Ramen,Ramen,223


In [150]:
yelp_df

,yelp_bike_station_coordinates,name,latitude,longitude,rating,price,distance,category_1,category_2,category_3,review_count
0,"49.262487,-123.114397",Saku,49.263101,-123.116675,4.5,$$,38.092128,Japanese,None,None,224
1,"49.262487,-123.114397",Yolks,49.263114,-123.116353,3.5,$$,29.354770,Coffee & Tea,Breakfast & Brunch,Breakfast & Brunch,348
2,"49.262487,-123.114397",Marulilu Cafe,49.263380,-123.114150,4.0,$$,153.789445,Cafes,None,None,286
3,"49.262487,-123.114397",La Taqueria Pinche Taco Shop,49.263559,-123.112736,4.0,$$,255.181363,Mexican,None,None,682
4,"49.262487,-123.114397",Hokkaido Ramen Santouka,49.263127,-123.116892,4.0,$$,50.822267,Noodles,Ramen,Ramen,223
...,...,...,...,...,...,...,...,...,...,...,...
4875,"49.264019,-123.209176",Menya Raizo,49.263360,-123.113140,4.0,$$,226.906974,Ramen,None,None,75
4876,"49.264019,-123.209176",Sushi California,49.263045,-123.112614,3.5,$$,263.365095,Japanese,Sushi Bars,Sushi Bars,317
4877,"49.264019,-123.209176",Passion8 Dessert Cafe,49.258435,-123.114998,4.0,$$,545.823128,Desserts,Korean,Korean,210
4878,"49.264019,-123.209176",Banana Leaf,49.263020,-123.122500,4.0,$$,454.459866,Malaysian,Singaporean,Singaporean,300


In [148]:
#Check the dimensions of the dataframe
print(yelp_df.shape)
#yelp_df

(4880, 11)


In [149]:
#Save the yelp_df as a csv file 
yelp_df.to_csv('yelp_data_latest1.csv', index=False)

# Comparing Results

In [96]:
#Remove duplicate rows from dataframe and drop bike_station_coordinates column.
Foursquare_df_clean= Foursquare_df.drop(columns=['bike_station_coordinates'])

#Remove duplicate rows
Foursquare_df_clean = Foursquare_df_clean.drop_duplicates(keep=False, inplace=False)

,name,category1,category2,category3,distance,latitude,longitude,locality,price,rating
0,Marulilu Cafe,Café,Japanese Restaurant,None,96,49.263328,-123.114020,Vancouver,2.0,8.2
1,Aperture Coffee Bar,Café,Coffee Shop,Restaurant,299,49.263292,-123.110529,Vancouver,1.0,8.7
2,Milano Coffee,Coffee Shop,Restaurant,None,430,49.263724,-123.108701,Vancouver,2.0,9.0
3,Caffe Cittadella,Coffee Shop,Restaurant,None,336,49.264730,-123.116978,Vancouver,2.0,8.6
4,Elysian Coffee Broadway,Coffee Shop,Restaurant,None,225,49.263077,-123.117243,Vancouver,1.0,8.1
...,...,...,...,...,...,...,...,...,...,...
2431,Mix the Bakery,Bakery,Pizzeria,Fast Food Restaurant,157,49.263658,-123.207055,Vancouver,1.0,6.7
2432,Subway,Deli,Fast Food Restaurant,None,303,49.263860,-123.204964,Vancouver,1.0,6.5
2433,Pizza Hut,Pizzeria,Restaurant,None,130,49.263728,-123.210465,Vancouver,1.0,6.3
2434,Gold Train Express,Vietnamese Restaurant,None,None,75,49.263702,-123.210139,Vancouver,2.0,6.0


In [99]:
#Reset index
Foursquare_df_clean.reset_index(drop=True)
Foursquare_df_clean.head()

,name,category1,category2,category3,distance,latitude,longitude,locality,price,rating
0,Marulilu Cafe,Café,Japanese Restaurant,None,96,49.263328,-123.114020,Vancouver,2.0,8.2
1,Aperture Coffee Bar,Café,Coffee Shop,Restaurant,299,49.263292,-123.110529,Vancouver,1.0,8.7
2,Milano Coffee,Coffee Shop,Restaurant,None,430,49.263724,-123.108701,Vancouver,2.0,9.0
3,Caffe Cittadella,Coffee Shop,Restaurant,None,336,49.264730,-123.116978,Vancouver,2.0,8.6
4,Elysian Coffee Broadway,Coffee Shop,Restaurant,None,225,49.263077,-123.117243,Vancouver,1.0,8.1


In [ ]:
Get the top 10 restaurants according to their rating

In [75]:
#Filtering categories to include only restaurants
Foursquare_top10 = Foursquare_df_clean.loc[(Foursquare_df_clean['category1'].str.contains('Restaurant'))|(Foursquare_df_clean['category2'].str.contains('Restaurant'))|(Foursquare_df_clean['category3'].str.contains('Restaurant'))]

#Fetching the top 10 restuarants by Rating and then by distance
Foursquare_top10= Foursquare_df_clean.sort_values(['rating', 'distance'], ascending=False)
Foursquare_top10.head(10)
Foursquare_top10.shape

(2409, 10)

In [151]:
#Remove duplicate rows from dataframe
yelp_df_clean = yelp_df_clean.drop_duplicates(keep=False, inplace=False)

In [152]:
#drop Distance column and bike station coordinates column
yelp_df_clean= yelp_df.drop(columns=['distance', 'yelp_bike_station_coordinates'])

In [153]:
#Remove duplicate rows from dataframe
#yelp_df_clean = yelp_df_clean.drop_duplicates(keep=False, inplace=False)

In [102]:
#Filtering categories to include only restuarants
yelp_top10 = yelp_df_clean.loc[(yelp_df_clean['category_1'].str.contains('Restaurant'))|(yelp_df_clean['category_2'].str.contains('Restaurant'))|(yelp_df_clean['category_3'].str.contains('Restaurant'))]

#Fetching the top 10 restuarants by Rating and then by popularity
yelp_top10= yelp_df_clean.sort_values(['rating'], ascending=False)
yelp_top10.head(10)

,name,latitude,longitude,rating,price,category_1,category_2,category_3,review_count
0,Saku,49.263101,-123.116675,4.5,$$,Japanese,None,None,224
2033,Indian Roti Kitchen,49.259010,-123.115380,4.5,$$,Indian,None,None,159
2080,Saku,49.263101,-123.116675,4.5,$$,Japanese,None,None,224
2073,Indian Roti Kitchen,49.259010,-123.115380,4.5,$$,Indian,None,None,159
2066,Uma Sushi,49.263805,-123.113729,4.5,$$$,Japanese,Sushi Bars,Sushi Bars,113
2060,Saku,49.263101,-123.116675,4.5,$$,Japanese,None,None,224
2053,Indian Roti Kitchen,49.259010,-123.115380,4.5,$$,Indian,None,None,159
2046,Uma Sushi,49.263805,-123.113729,4.5,$$$,Japanese,Sushi Bars,Sushi Bars,113
2040,Saku,49.263101,-123.116675,4.5,$$,Japanese,None,None,224
2026,Uma Sushi,49.263805,-123.113729,4.5,$$$,Japanese,Sushi Bars,Sushi Bars,113


In [56]:
#drop Distance column and bike station coordinates column
yelp_df_clean= yelp_df.drop(columns=['distance', 'yelp_bike_station_coordinates'])

In [57]:
#Remove duplicate rows from dataframe
yelp_df_clean = yelp_df_clean.drop_duplicates(keep=False, inplace=False)

In [104]:
yelp_df_clean.head()

,name,latitude,longitude,rating,price,category_1,category_2,category_3,review_count
0,Saku,49.263101,-123.116675,4.5,$$,Japanese,None,None,224
1,La Taqueria Pinche Taco Shop,49.263559,-123.112736,4.0,$$,Mexican,None,None,682
2,Yolks,49.263114,-123.116353,3.5,$$,Coffee & Tea,Breakfast & Brunch,Breakfast & Brunch,348
3,Hokkaido Ramen Santouka,49.263127,-123.116892,4.0,$$,Noodles,Ramen,Ramen,223
4,Rogue Kitchen & Wetbar,49.263047,-123.117827,3.5,$$,American (Traditional),Gastropubs,Gastropubs,146


Which API provided you with more complete data? Provide an explanation. 

In [ ]:
1. Yelp Fusion API:
    
    - The Yelp API has a user-friendly experience with clear documentation.
    - It focused on local businesses and provides valuable insights for location-specific information, 
      such as distance from the bike station, rating etc. It also has rich content, including photos and menus. 

2. FourSquare API:
    
    - Foursquare API had very easy API access and fields were pretty self explanatory
    - Also, it gives all the available business within the given radius 

Comparison Result:
     I performed couple of testing on various categories using the Foursquare and Yelp API. Here are my findings:
        - For the categories such as Travel and Transportation, I tapped the Foursquare API and got the details 
          such as the near by travel hotels and transportation options. 
        - Foursquare provided me less options from travel lodges and transportation perspective. On the other hand
          Yelp API provided very detailed information such as location, latitude, longitude, ratings etc. I felt 
            there is more complete data from Yelp when compared to that of foursquare API
        - Also, when I did a different category such as Parks, Restaurants and ATM, FoursquareAPI had more data.
        - It was easy to fetch data based on a particular given location in Foursquare API on contrary to Yelp where
           we would need to pass on the latitude and longitude co-ordinates.
        - Also, Yelp retrieved very less details for the subcategories of ATM, rather it was able to provide more data
            for the general category search of Banks.
        - In addition to this, sub category specific details such as distance, rating etc were missing in Yelp

Overall Comparison Results:
    On the whole, based on the categories that we select the API outcomes would be either more or with less details.
    Foursquare API has advantages to pass location as param to identify the required POI near the area of our interest.
    But with Yelp, we had to provide specific co-ordinates and get details. However Yelp provides robust data based on
    the values we pass. In addition to the generic details from foursquare, I was able to fetch ratings from Yelp which 
    would give it an upper hand over foursquare for certain use case evaluations.

Also, I have provided the top values from both the Yelp and Foursquare data sets for further better understanding.